In [1]:
import requests
import json
import datetime
from datetime import datetime
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
#from ETH_parser import get_floor_price, get_sales_collection, get_sales_token, get_attributes, get_attributes_collection, get_rarity, get_block_timestamp

In [2]:
API_KEY = "vYHU2nVXBMrIHlByeFOQWubnRQcl_v_3"
BAYC = "0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D"

In [3]:
def get_floor_price(API: str, address: str) -> dict:
    url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{API}/getFloorPrice?contractAddress={address}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    
    prices = {"opensea": data['openSea']['floorPrice'],
              "looksrare": data['looksRare']['floorPrice']}

    return prices

In [4]:
# OLD
# def get_sales_collection(API: str, address: str, limit: int) -> json:
#     url = f"https://eth-mainnet.g.alchemy.com/nft/v3/{API}/getNFTSales?fromBlock=0&toBlock=latest&order=desc&marketplace=seaport&contractAddress={address}&limit={limit}"
#     headers = {"accept": "application/json"}
#     response = requests.get(url, headers=headers)
#     data = json.loads(response.text)

    
#     activity = [{"tokenId": sale["tokenId"],
#                  "marketplace": sale["marketplace"],
#                  "block": sale["blockNumber"],
#                  "token": sale["sellerFee"]["symbol"],
#                  "price": 0 if sale["sellerFee"]["amount"] is None else int(sale["sellerFee"]["amount"]) / 10 ** 18} for sale in data["nftSales"]]

#     return activity

In [ ]:
def get_sales_collection(API: str, address: str, limit: int):
    url = f"https://eth-mainnet.g.alchemy.com/nft/v3/{API}/getNFTSales?fromBlock=0&toBlock=latest&order=desc&marketplace=seaport&contractAddress={address}&limit={limit}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)

    activity = [{"tokenId": sale["tokenId"],
                 "marketplace": sale["marketplace"],
                 "block": sale["blockNumber"],
                 "token": sale["sellerFee"]["symbol"],
                 "price": 0 if sale["sellerFee"]["amount"] is None else int(sale["sellerFee"]["amount"]) / 10 ** 18} for sale in data["nftSales"]]

    return pd.DataFrame(activity)

In [5]:
def get_sales_token(API: str, address: str, limit: int, tokenId: int) -> json:
    url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{API}/getNFTSales?fromBlock=0&toBlock=latest&order=desc&contractAddress={address}&limit={limit}&tokenId={tokenId}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    
    activity = [{"tokenId": sale["tokenId"],
                 "marketplace": sale["marketplace"],
                 "sellerFee": int(sale["sellerFee"]["amount"]) / 10 ** 18} for sale in data["nftSales"]]
    
    return activity

In [6]:
def get_attributes(API: str, address: str, tokenId: int) -> json:
    url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{API}/computeRarity?contractAddress={address}&tokenId={tokenId}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)

    return data

In [7]:
def get_attributes_collection(API: str, address: str) -> json:
    url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{API}/summarizeNFTAttributes?contractAddress={address}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    
    return data['summary']

In [8]:
def get_rarity(API: str, address: str, tokenId: int) -> json:
    data = get_attributes(API=API, address=address, tokenId=tokenId)
    
    result = 1
    for rarity in data:
        result *= rarity["prevalence"]
        
    return result

In [16]:
def get_block_timestamp(block_number):
    url = f"https://eth-mainnet.alchemyapi.io/v2/{API_KEY}"
    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "eth_getBlockByNumber",
        "params": [hex(block_number), False],
        "id": 1
    })
    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code == 200:
        timestamp = response.json()['result']['timestamp']
        date_time = datetime.utcfromtimestamp(int(timestamp, 16))
        return date_time
    else:
        return "error"

In [19]:
sales = get_sales_collection(API=API_KEY, address=BAYC, limit=3)

In [20]:
sales['date'] = sales['block'].apply(get_block_timestamp)
sales

,tokenId,marketplace,block,token,price,date
0,4871,seaport,17415238,ETH,1.24,2023-06-05 15:35:47
1,8230,seaport,17361994,,0.00,2023-05-29 03:31:59
2,7851,seaport,17295760,,0.00,2023-05-19 19:59:47
